In [1]:
import sqlite3
import pandas as pd

In [2]:
conn=sqlite3.connect('database.db')

In [3]:
cur=conn.cursor()

In [4]:
cur.execute("""
CREATE TABLE customers (
    customer_id   INTEGER PRIMARY KEY,
    first_name    TEXT NOT NULL,
    last_name     TEXT NOT NULL,
    email         TEXT UNIQUE NOT NULL,
    city          TEXT,
    country       TEXT
);
""")

cur.execute("""
CREATE TABLE products (
    product_id    INTEGER PRIMARY KEY,
    product_name  TEXT NOT NULL,
    category      TEXT,
    unit_price    REAL NOT NULL  -- DECIMAL -> REAL u SQLite
);
""")

cur.execute("""
CREATE TABLE orders (
    order_id      INTEGER PRIMARY KEY,
    customer_id   INTEGER NOT NULL,
    order_date    DATE NOT NULL,
    status        TEXT NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
""")


cur.execute("""
CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY,
    order_id      INTEGER NOT NULL,
    product_id    INTEGER NOT NULL,
    quantity      INTEGER NOT NULL,
    item_price    REAL NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
""")

In [6]:
customers_data = [
    (1, 'Ana', 'Horvat', 'ana.horvat@example.com', 'Zagreb', 'Croatia'),
    (2, 'Marko', 'Kovač', 'marko.kovac@example.com', 'Rijeka', 'Croatia'),
    (3, 'Ivana', 'Marić', 'ivana.maric@example.com', 'Split', 'Croatia'),
    (4, 'Petar', 'Novak', 'petar.novak@example.com', 'Ljubljana', 'Slovenia'),
    (5, 'Luka', 'Babić', 'luka.babic@example.com', 'Beč', 'Austria'),
    (6, 'Maja', 'Jurić', 'maja.juric@example.com', 'Osijek', 'Croatia')
]
cur.executemany("INSERT INTO customers VALUES (?, ?, ?, ?, ?, ?)", customers_data)

In [7]:
cur.execute("SELECT first_name FROM customers").fetchall()

[('Ana',), ('Marko',), ('Ivana',), ('Petar',), ('Luka',), ('Maja',)]

In [9]:
cur.execute("SELECT first_name, last_name,city FROM customers WHERE city LIKE 'Z%'").fetchall()

[('Ana', 'Horvat', 'Zagreb')]

In [10]:
cur.execute("SELECT first_name,country FROM customers WHERE country LIKE 'C%'").fetchall()

[('Ana', 'Croatia'),
 ('Marko', 'Croatia'),
 ('Ivana', 'Croatia'),
 ('Maja', 'Croatia')]

In [12]:
products_data = [
    (1, 'Laptop 14\"', 'Elektronika', 750.00),
    (2, 'Bežične slušalice', 'Elektronika', 120.00),
    (3, 'Mehanička tipkovnica', 'Elektronika', 90.00),
    (4, 'Uredsko stolac', 'Namještaj', 150.00),
    (5, 'Radni stol', 'Namještaj', 220.00),
    (6, 'Bilježnica A4', 'Papirnica', 4.50),
    (7, 'Hemijska olovka', 'Papirnica', 1.20),
    (8, 'Monitor 24\"', 'Elektronika', 180.00)
]
cur.executemany("INSERT INTO products VALUES (?, ?, ?, ?)", products_data)

In [13]:
orders_data = [
    (1, 1, '2025-11-15', 'shipped'),
    (2, 1, '2025-12-02', 'shipped'),
    (3, 2, '2025-12-20', 'pending'),
    (4, 3, '2026-01-05', 'shipped'),
    (5, 3, '2026-01-18', 'cancelled'),
    (6, 4, '2025-10-10', 'shipped'),
    (7, 5, '2025-09-25', 'shipped'),
    (8, 6, '2026-01-22', 'pending'),
    (9, 2, '2026-01-10', 'shipped'),
    (10, 1, '2026-01-23', 'shipped')
]
cur.executemany("INSERT INTO orders VALUES (?, ?, ?, ?)", orders_data)

In [15]:
order_items_data = [
    (1, 1, 1, 1, 750.00),
    (2, 1, 2, 1, 115.00),
    (3, 2, 6, 5, 4.50),
    (4, 2, 7, 10, 1.10),
    (5, 3, 3, 1, 90.00),
    (6, 3, 8, 2, 175.00),
    (7, 4, 1, 1, 730.00),
    (8, 4, 2, 1, 120.00),
    (9, 5, 4, 2, 150.00),
    (10, 5, 5, 1, 220.00),
    (11, 6, 8, 1, 180.00),
    (12, 6, 6, 3, 4.40),
    (13, 7, 2, 2, 120.00),
    (14, 7, 3, 1, 95.00),
    (15, 8, 5, 1, 220.00),
    (16, 8, 6, 10, 4.50),
    (17, 9, 1, 1, 760.00),
    (18, 9, 7, 20, 1.20),
    (19, 10, 8, 1, 185.00),
    (20, 10, 2, 1, 118.00)
]
cur.executemany("INSERT INTO order_items VALUES (?, ?, ?, ?, ?)", order_items_data)

In [16]:
import pandas as pd
from pandasql import sqldf

In [17]:
customers = pd.read_sql_query("SELECT * FROM customers", conn)
products = pd.read_sql_query("SELECT * FROM products", conn)
orders = pd.read_sql_query("SELECT * FROM orders", conn)
order_items = pd.read_sql_query("SELECT * FROM order_items", conn)

In [18]:
conn.close()

In [19]:
pysqldf = lambda q: sqldf(q, globals())

In [21]:
customers

,customer_id,first_name,last_name,email,city,country
0,1,Ana,Horvat,ana.horvat@example.com,Zagreb,Croatia
1,2,Marko,Kovač,marko.kovac@example.com,Rijeka,Croatia
2,3,Ivana,Marić,ivana.maric@example.com,Split,Croatia
3,4,Petar,Novak,petar.novak@example.com,Ljubljana,Slovenia
4,5,Luka,Babić,luka.babic@example.com,Beč,Austria
5,6,Maja,Jurić,maja.juric@example.com,Osijek,Croatia


In [24]:
products

,product_id,product_name,category,unit_price
0,1,"Laptop 14""",Elektronika,750.0
1,2,Bežične slušalice,Elektronika,120.0
2,3,Mehanička tipkovnica,Elektronika,90.0
3,4,Uredsko stolac,Namještaj,150.0
4,5,Radni stol,Namještaj,220.0
5,6,Bilježnica A4,Papirnica,4.5
6,7,Hemijska olovka,Papirnica,1.2
7,8,"Monitor 24""",Elektronika,180.0


In [32]:
pysqldf("""SELECT c.first_name, c.last_name, COUNT(o.status) 
FROM customers c  JOIN orders o ON c.customer_id=o.customer_id
WHERE o.status='shipped'
GROUP BY c.first_name, c.last_name""")

,first_name,last_name,COUNT(o.status)
0,Ana,Horvat,3
1,Ivana,Marić,1
2,Luka,Babić,1
3,Marko,Kovač,1
4,Petar,Novak,1


In [25]:
orders

,order_id,customer_id,order_date,status
0,1,1,2025-11-15,shipped
1,2,1,2025-12-02,shipped
2,3,2,2025-12-20,pending
3,4,3,2026-01-05,shipped
4,5,3,2026-01-18,cancelled
5,6,4,2025-10-10,shipped
6,7,5,2025-09-25,shipped
7,8,6,2026-01-22,pending
8,9,2,2026-01-10,shipped
9,10,1,2026-01-23,shipped


In [27]:
order_items

,order_item_id,order_id,product_id,quantity,item_price
0,1,1,1,1,750.0
1,2,1,2,1,115.0
2,3,2,6,5,4.5
3,4,2,7,10,1.1
4,5,3,3,1,90.0
5,6,3,8,2,175.0
6,7,4,1,1,730.0
7,8,4,2,1,120.0
8,9,5,4,2,150.0
9,10,5,5,1,220.0
